# DATA

In [1]:
import numpy as np
import pandas as pd

In [12]:
vendas = pd.read_csv('../../../data/interim/Vendas.csv', encoding='ISO-8859-1')

In [13]:
vendas = vendas.replace(['ST', 'CORSÁRIO SUPLEX', 'ESP', 'ESPECIAL'], ['00','CORSARIO SUPLEX', 'GG', 'GG'])

In [14]:
a = vendas.groupby(['ANO', 'MES', 'DESCRICAO', 'NOME', 'TAMANHO'])

In [15]:
df = a.sum().reset_index().drop(['DIA', 'VLR_UNIT'], 1)
df = df.sort_values(by=['ANO','MES', 'DESCRICAO', 'TAMANHO', 'NOME'])

In [66]:
vendas.groupby(['DESCRICAO', 'ANO', 'TAMANHO']).mean()['VLR_UNIT']

DESCRICAO                  ANO     TAMANHO
BERMUDA CICLISTA COLEGIAL  2016.0  8           57.500000
                                   9           57.500000
                                   10          57.500000
                                   11          57.500000
                           2017.0  8           63.900000
                                   9           64.000000
                                   10          64.000000
                                   11          64.000000
                           2018.0  8           70.000000
                                   9           70.000000
BERMUDA CICLISTA COTTON    2016.0  1           51.000000
                                   2           51.000000
                                   3           51.307692
                                   4           51.000000
                           2017.0  0           57.000000
                                   1           57.000000
                                   2         

In [64]:
vendas[(vendas.ANO == 2016) & (vendas.DESCRICAO == 'BERMUDA CICLISTA COTTON')]

,ANO,DESCRICAO,DIA,HORA,MES,NOME,QUANTIDADE,TAMANHO,VLR_UNIT
149,2016.0,BERMUDA CICLISTA COLEGIAL,13.0,13:21,1.0,ITAIM,1.0,8,57.5
833,2016.0,BERMUDA CICLISTA COLEGIAL,19.0,07:56,1.0,ALDEIA DA SERRA,1.0,9,57.5
1044,2016.0,BERMUDA CICLISTA COLEGIAL,20.0,10:34,1.0,VERBO DIVINO,1.0,9,57.5
1067,2016.0,BERMUDA CICLISTA COLEGIAL,20.0,11:26,1.0,VERBO DIVINO,1.0,8,57.5
1093,2016.0,BERMUDA CICLISTA COLEGIAL,20.0,14:15,1.0,VERBO DIVINO,1.0,10,57.5
2020,2016.0,BERMUDA CICLISTA COLEGIAL,28.0,14:25,1.0,ALDEIA DA SERRA,1.0,8,57.5
4494,2016.0,BERMUDA CICLISTA COLEGIAL,10.0,11:05,3.0,VERBO DIVINO,1.0,11,57.5
7386,2016.0,BERMUDA CICLISTA COLEGIAL,19.0,10:08,10.0,VERBO DIVINO,1.0,10,57.5


In [56]:
df = df.merge(vendas['ANO', 'MES', 'DESCRICAO', 'TAMANHO', 'NOME', 'VLR_UNIT'], 
              on=['ANO', 'MES', 'DESCRICAO', 'TAMANHO', 'NOME'],
             how='left')

KeyError: ('ANO', 'MES', 'DESCRICAO', 'TAMANHO', 'NOME', 'VLR_UNIT')

In [16]:
treino = df[df.ANO < 2018]
validacao = df[df.ANO >= 2018]

In [54]:
vendas.head()

,ANO,DESCRICAO,DIA,HORA,MES,NOME,QUANTIDADE,TAMANHO,VLR_UNIT
0,2016.0,CAMISETA MANGA CURTA,12.0,14:54,1.0,VERBO DIVINO,7.0,0,35.5
1,2016.0,BERMUDA CICLISTA COTTON,12.0,15:08,1.0,VERBO DIVINO,1.0,4,51.0
2,2016.0,CAMISETA MANGA CURTA,12.0,15:08,1.0,VERBO DIVINO,5.0,8,37.5
3,2016.0,BLUSA MOLETOM FLANELADA,12.0,15:27,1.0,VERBO DIVINO,1.0,10,115.0
4,2016.0,CALÇA SARJA MASC,12.0,15:27,1.0,VERBO DIVINO,1.0,12,116.0


In [55]:
df.head()

,ANO,MES,DESCRICAO,NOME,TAMANHO,QUANTIDADE
3,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,VERBO DIVINO,M,1.0
0,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,ALDEIA DA SERRA,P,1.0
4,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,VERBO DIVINO,P,1.0
1,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,ALDEIA DA SERRA,PP,1.0
2,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,ITAIM,PP,1.0


In [18]:
treino.shape, validacao.shape

((4396, 6), (985, 6))

In [19]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [20]:
le_DESC = LabelEncoder()
treino['DESCRICAO'] = le_DESC.fit_transform(treino.DESCRICAO)
validacao['DESCRICAO'] = le_DESC.transform(validacao.DESCRICAO)

/Users/chazanas/.virtualenvs/sold_alt/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/chazanas/.virtualenvs/sold_alt/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
tamanho_val = {
    '00' : 0,
    '02' : 1,
    '04' : 2,
    '06' : 3,
    '08' : 4,
    '10' : 5,
    '12' : 6,
    '14' : 7,
    'PP' : 8,
    'P' : 9,
    'M' : 10,
    'G' : 11,
    'GG' : 12
}
treino.TAMANHO = treino.TAMANHO.apply(lambda x: tamanho_val[x])
validacao.TAMANHO = validacao.TAMANHO.apply(lambda x: tamanho_val[x])

/Users/chazanas/.virtualenvs/sold_alt/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [28]:
le_NOME = LabelEncoder()
treino['NOME'] = le_NOME.fit_transform(treino.NOME)
validacao['NOME'] = le_NOME.transform(validacao.NOME)

/Users/chazanas/.virtualenvs/sold_alt/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/chazanas/.virtualenvs/sold_alt/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [51]:
y_train = treino['QUANTIDADE']
X_train = treino.drop('QUANTIDADE', 1)
X_test = validacao.drop('QUANTIDADE', 1)
Y_test = validacao['QUANTIDADE']

# FEATURES

# TRAIN

In [48]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from xgboost import XGBClassifier

In [49]:
models = []
models.append(('LR', LogisticRegression()))
models.append(('RF', RandomForestRegressor()))
models.append(('XTREE', ExtraTreesRegressor()))
models.append(('GBR', GradientBoostingRegressor()))
models.append(('XGB', XGBClassifier()))
results = []
for name, model in models:
    model.fit(X_train, y_train)
    Y_pred = model.predict(X_test)
    print(name, mean_squared_error(Y_test, Y_pred))

LR 226.55837563451777
RF 56.31938071065989
XTREE 62.91501522842639
GBR 82.76824095987064
XGB 154.7055837563452


/Users/chazanas/.virtualenvs/sold_alt/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# RANDOM FOREST